<a href="https://colab.research.google.com/github/chobocoding/deeplearning/blob/main/doghole_keeper(transfer%20learning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !apt install unzip
# !unzip hw5_doghole_keeper.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/IT융합/hw5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import glob
from PIL import Image
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms as T
import torchvision.models as models

### 1. Data preparation

In [ ]:
transform = T.Compose([T.Resize(256), T.RandomCrop(224), T.ToTensor()])

In [ ]:
class Dataset(Dataset):
    def __init__(self, path='train'):
        super(Dataset, self).__init__()

        self.img_path = []
        bo_path = glob.glob(path+'/bo/*.jpg')
        notbo_path = glob.glob(path+'/not_bo/*.jpg')
        self.img_path = bo_path + notbo_path

        self.label_list = [1]*len(bo_path) + [0]*len(notbo_path)

    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])
        img_pil = Image.fromarray(img)
        self.img_tensor = transform(img_pil)
        self.label_tensor = torch.tensor(self.label_list[index])
        return self.img_tensor.to('cuda:0'), self.label_tensor.to('cuda:0')


    def __len__(self):
        return len(self.img_path)

In [ ]:
training_dataset = Dataset('train')
training_loader = DataLoader(dataset=training_dataset, batch_size=8, shuffle=True)

In [ ]:
validation_dataset = Dataset('valid')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=8, shuffle=False)

### 2. Constructing a convolutional neural network

In [ ]:
vgg16 = models.vgg16(pretrained=True)

In [ ]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
vgg16.classifier[0].in_features

25088

In [ ]:
# freeze
for param in vgg16.parameters():
    param.requires_grad = False

In [ ]:
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 2)
)

In [ ]:
vgg16 = vgg16.to('cuda:0')

### 3. Loss function and optimization method

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.0005)

### 4. Training of a neural network

In [ ]:
def val_acc():
    pred_list = []
    label_list = []

    for itr, data in enumerate(validation_loader):
        inputs, labels = data

        pred = vgg16(inputs)
        pred_category = torch.argmax(pred, dim=1)

        pred_list = pred_list + list(pred_category)
        label_list = label_list + list(labels)

    accu = np.mean( np.array(pred_list) == np.array(label_list) )
    print("     Validation accuracy:", accu)
    return None

In [ ]:
for epoch in range(6):
    loss_val = 0
    for itr, data in enumerate(training_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = vgg16(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)
    if (epoch+1) % 2 == 0: val_acc()

Epoch: 1   , Loss: 4.647249927744269
Epoch: 2   , Loss: 1.1398980850353837
     Validation accuracy: 0.9666666666666667
Epoch: 3   , Loss: 0.734001649543643
Epoch: 4   , Loss: 0.3361550129484385
     Validation accuracy: 0.9666666666666667
Epoch: 5   , Loss: 0.24052892869804054
Epoch: 6   , Loss: 0.41657176171429455
     Validation accuracy: 1.0


### 5. Prediction and evluation for the validation set

In [ ]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred = vgg16(inputs)
    pred_category = torch.argmax(pred, dim=1)
    pred_list = pred_list + list(pred_category)
    label_list = label_list + list(labels)
accu = np.mean( np.array(pred_list) == np.array(label_list) )
print("Validation accuracy:", accu)

Validation accuracy: 0.9333333333333333


In [ ]:
pred_list

[tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0')]

In [ ]:
pred_list

[tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0')]